In [1]:
from common import Trial
from typing import List
from tqdm import tqdm
import re
import traceback
import numpy as np
import os
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt



Using TensorFlow backend.


In [2]:
channel_map = {'FP1': 0,
 'FP2': 1,
 'F7': 2,
 'F8': 3,
 'AF1': 4,
 'AF2': 5,
 'FZ': 6,
 'F4': 7,
 'F3': 8,
 'FC6': 9,
 'FC5': 10,
 'FC2': 11,
 'FC1': 12,
 'T8': 13,
 'T7': 14,
 'CZ': 15,
 'C3': 16,
 'C4': 17,
 'CP5': 18,
 'CP6': 19,
 'CP1': 20,
 'CP2': 21,
 'P3': 22,
 'P4': 23,
 'PZ': 24,
 'P8': 25,
 'P7': 26,
 'PO2': 27,
 'PO1': 28,
 'O2': 29,
 'O1': 30,
 'X': 31,
 'AF7': 32,
 'AF8': 33,
 'F5': 34,
 'F6': 35,
 'FT7': 36,
 'FT8': 37,
 'FPZ': 38,
 'FC4': 39,
 'FC3': 40,
 'C6': 41,
 'C5': 42,
 'F2': 43,
 'F1': 44,
 'TP8': 45,
 'TP7': 46,
 'AFZ': 47,
 'CP3': 48,
 'CP4': 49,
 'P5': 50,
 'P6': 51,
 'C1': 52,
 'C2': 53,
 'PO7': 54,
 'PO8': 55,
 'FCZ': 56,
 'POZ': 57,
 'OZ': 58,
 'P2': 59,
 'P1': 60,
 'CPZ': 61,
 'nd': 62,
 'Y': 63}

In [3]:
with open('eeg_full/co2a0000364.txt', 'r') as f:
    file_content = f.read()
    
print(file_content[:200])


# co2a0000364.rd
# 120 trials, 64 chans, 416 samples 368 post_stim samples
# 3.906000 msecs uV
# S1 obj , trial 0
# FP1 chan 0
0 FP1 0 -8.921
0 FP1 1 -8.433
0 FP1 2 -2.574
0 FP1 3 5.239
0 FP1 4 11.587


In [4]:
trials = list(filter(lambda string: len(string) > 0, re.split(r'^# co\w{9}.rd', file_content, flags=re.MULTILINE)))
trials[0]

trial = trials[0]
subject_id = file_content[2:13]
if 'a' == subject_id[3]:
    subject_class = 1
elif 'c' == subject_id[3]:
    subject_class = 0
else:
    raise ValueError('Invalid subject class: ' + subject_id)
    
lines = trial.split('\n')
trial_type_str, trial_number = tuple(lines[3].split(' , trial'))
trial_number = int(trial_number)

if trial_type_str == '# S1 obj':
    trial_type = 0
elif trial_type_str == '# S2 match':
    trial_type = 1
elif trial_type_str == '# S2 nomatch':
    trial_type = 2
else:
    raise ValueError('Invalid trial_type_str: ' + trial_type_str)

lines = lines[4:]


In [5]:
measurements = [(channel_map[line.split()[1]], int(line.split()[2]), float(line.split()[3])) for line in lines if len(line) > 0 and line[0] != '#']
measurements = np.array(measurements)

print(measurements)

[[  0.      0.     -8.921]
 [  0.      1.     -8.433]
 [  0.      2.     -2.574]
 ...
 [ 63.    253.     15.849]
 [ 63.    254.     16.337]
 [ 63.    255.     14.872]]


In [6]:
eeg = measurements[:, 2].reshape((64,256))
eeg = np.array([np.mean(eeg_channel.reshape(-1, 4), axis=1) for eeg_channel in eeg[::1]])
eeg = (eeg - np.min(eeg))/np.ptp(eeg)

In [7]:
eeg

array([[0.23834638, 0.4916502 , 0.28900715, ..., 0.48953628, 0.3607726 ,
        0.39665749],
       [0.37748516, 0.44503141, 0.25927383, ..., 0.46825435, 0.33949067,
        0.39436632],
       [0.23095412, 0.50536695, 0.29006195, ..., 0.59612749, 0.31960938,
        0.43992599],
       ...,
       [0.28725202, 0.29780005, 0.27880064, ..., 0.36534197, 0.35268434,
        0.3674559 ],
       [0.23658694, 0.49199604, 0.2872477 , ..., 0.49199604, 0.36112708,
        0.40545471],
       [0.29727264, 0.46614042, 0.30360578, ..., 0.67088876, 0.4133614 ,
        0.55901488]])

In [8]:
Trial(subject_id, subject_class, trial_number, trial_type, eeg)

{
    "subject_id": "co2a0000364",
    "subject_class": 1,
    "trial_number": 0,
    "trial_type": 0,
    "eeg": "[[0.23834638 0.4916502  0.28900715 ... 0.48953628 0.3607726  0.39665749]\n [0.37748516 0.44503141 0.25927383 ... 0.46825435 0.33949067 0.39436632]\n [0.23095412 0.50536695 0.29006195 ... 0.59612749 0.31960938 0.43992599]\n ...\n [0.28725202 0.29780005 0.27880064 ... 0.36534197 0.35268434 0.3674559 ]\n [0.23658694 0.49199604 0.2872477  ... 0.49199604 0.36112708 0.40545471]\n [0.29727264 0.46614042 0.30360578 ... 0.67088876 0.4133614  0.55901488]]"
}

In [9]:
errors = 0
zeros = 0

def read_trials(eeg_file: str) -> List[Trial]:
    global errors, zeros
    
    with open(eeg_file, 'r') as f:
        file_content = f.read()
        
        subject_id = file_content[2:13]
        if 'a' == subject_id[3]:
            subject_class = 1
        elif 'c' == subject_id[3]:
            subject_class = 0
        else:
            raise ValueError('Invalid subject class: ' + subject_id)
        
        trials = []
        trials_str = list(filter(lambda string: len(string) > 0, re.split(r'^# co\w{9}.rd', file_content, flags=re.MULTILINE)))

        
        for trial in trials_str:
            try:
                lines = list(filter(lambda line: len(line) > 0, map(lambda line: line.strip(), trial.split('\n'))))
                trial_type_str, trial_number = tuple(lines[2].split(', trial'))
                trial_number = int(trial_number)

                if 'err' in trial_type_str:
                    print('Skipping trial |{}| from file |{}| due to error type'.format(trial_number, eeg_file))
                    errors = errors + 1
                    continue
                elif trial_type_str.startswith('# S1 obj'):
                    trial_type = 0
                elif trial_type_str.startswith('# S2 match'):
                    trial_type = 1
                elif trial_type_str.startswith('# S2 nomatch'):
                    trial_type = 2
                else:
                    raise ValueError('Invalid trial_type_str: ' + trial_type_str)

                lines = lines[4:]

                measurements = [(channel_map[line.split()[1]], int(line.split()[2]), float(line.split()[3])) for line in lines if len(line) > 0 and line[0] != '#']
                measurements = np.array(measurements)
                eeg = measurements[:, 2].reshape((64,256))
                eeg = np.array([np.mean(eeg_channel.reshape(-1, 4), axis=1) for eeg_channel in eeg[::1]])
                eeg = (eeg - np.min(eeg))/np.ptp(eeg)
                

                if np.count_nonzero(eeg) == 0:
                    print('Skipping trial |{}| from file |{}| due to only 0 values'.format(trial_number, eeg_file))
                    zeros = zeros + 1
                    continue
                    
                trials.append(Trial(subject_id, subject_class, trial_number, trial_type, eeg))
            except:
                print(traceback.format_exc())
                print('Error in file: |{}|'.format(eeg_file))
                print('Error for: |{}|'.format(trial))
                print('Error for: |{}|'.format(lines[2]))
                raise
                
            
        return trials


In [10]:
all_trials = []

for file in tqdm(os.listdir('eeg_full')):
    all_trials = all_trials + read_trials('eeg_full/' + file)
    
print('Good trials: {}, Error trials: {}, Zeros trials: {}'.format(len(all_trials), errors, zeros))

  2%|▏         | 2/122 [00:26<24:30, 12.26s/it]

Skipping trial |89| from file |eeg_full/co2c0000362.txt| due to error type


  4%|▍         | 5/122 [01:05<25:26, 13.05s/it]

Skipping trial |5| from file |eeg_full/co2c0000348.txt| due to error type
Skipping trial |109| from file |eeg_full/co2c0000348.txt| due to error type


 10%|▉         | 12/122 [02:21<20:04, 10.95s/it]

...

Skipping trial |51| from file |eeg_full/co2c0000390.txt| due to error type


 98%|█████████▊| 120/122 [25:33<00:18,  9.24s/it]

Skipping trial |93| from file |eeg_full/co2c0000346.txt| due to error type


100%|██████████| 122/122 [25:52<00:00,  8.94s/it]

Good trials: 10962, Error trials: 95, Zeros trials: 0


In [11]:
df = pd.DataFrame.from_records([trial.to_dict() for trial in all_trials])
df = df.sample(frac=1).reset_index(drop=True)

In [12]:
sys.getsizeof(df)

361526864

In [13]:
df.head()

,eeg,subject_class,subject_id,trial_number,trial_type
0,"[[0.32602052002063964, 0.3249387123277357, 0.3...",0,co2c0000356,67,2
1,"[[0.2363887745245523, 0.24798597805260408, 0.2...",1,co2a0000445,79,1
2,"[[0.5380679338420093, 0.4866284407121465, 0.49...",1,co2a0000369,0,0
3,"[[0.4241038260292235, 0.5077221399627989, 0.51...",1,co2a0000394,71,1
4,"[[0.6637111720074131, 0.6888493766929085, 0.68...",1,co2a0000433,85,1


In [14]:
df.describe()

,subject_class,trial_number,trial_type
count,10962.000000,10962.000000,10962.000000
mean,0.634465,60.611294,0.749225
std,0.481602,34.155754,0.828051
min,0.000000,0.000000,0.000000
25%,0.000000,32.000000,0.000000
50%,1.000000,61.000000,1.000000
75%,1.000000,90.000000,1.000000
max,1.000000,119.000000,2.000000


In [15]:
X = df['eeg'].values
y = df['subject_class'].values

# keras required format
X = np.rollaxis(np.dstack(X), -1)
X = X.reshape(X.shape[0], 64, 64, 1)
y = keras.utils.to_categorical(y, 2)

In [16]:
X.shape

(10962, 64, 64, 1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [18]:
input_shape = (64, 64, 1)
num_classes = 2
batch_size=128
epochs=50

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.


In [19]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              10817000  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2002      
Total para

In [20]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('val_acc'))
        
history = AccuracyHistory()


In [21]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[history])


score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

plt.plot(range(1,len(history.acc) + 1), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

Instructions for updating:
Use tf.cast instead.
Train on 9865 samples, validate on 1097 samples
Epoch 1/50
9865/9865 [==============================] - 138s 14ms/step - loss: 0.6687 - acc: 0.6382 - val_loss: 0.5998 - val_acc: 0.6910
Epoch 2/50
9865/9865 [==============================] - 141s 14ms/step - loss: 0.5768 - acc: 0.7070 - val_loss: 0.5568 - val_acc: 0.7138
Epoch 3/50
9865/9865 [==============================] - 148s 15ms/step - loss: 0.5441 - acc: 0.7258 - val_loss: 0.5610 - val_acc: 0.7037
Epoch 4/50
9865/9865 [==============================] - 144s 15ms/step - loss: 0.5159 - acc: 0.7482 - val_loss: 0.6393 - val_acc: 0.6955
Epoch 5/50
9865/9865 [==============================] - 131s 13ms/step - loss: 0.4926 - acc: 0.7584 - val_loss: 0.5261 - val_acc: 0.7521
Epoch 6/50
9865/9865 [==============================] - 466s 47ms/step - loss: 0.4658 - acc: 0.7734 - val_loss: 0.4695 - val_acc: 0.7730
Epoch 7/50
9865/9865 [==============================] - 144s 15ms/step - loss: 0.4

KeyboardInterrupt: 

In [ ]:

plt.plot(range(1,len(history.acc) + 1), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()